## Task 1

Starting a Spark session

In [3]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
import matplotlib.pyplot as plt

appName = "CloudMQTT"
master = "local[*]"

conf = pyspark.SparkConf()\
    .set('spark.jars','gs://mqtt-bucket-14/postgresql-42.6.0.jar')\
    .setAppName(appName)\
    .setMaster(master)
# spark = SparkSession.builder \
#     .master("local[*]") \
#     .appName("CloudMQTT") \
#     .config('spark.jars', 'gs://mqtt-bucket-14/postgresql-42.6.0.jar')\
#     .getOrCreate()

# Create Spark Context with the new configurations rather than relying on the default one
sc = SparkContext.getOrCreate(conf=conf)
# You need to create SQL Context to conduct some database operations like what we will see later.
sqlContext = SQLContext(sc)

# If you have SQL context, you create the session from the Spark Context
spark = sqlContext.sparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/16 03:25:10 INFO SparkEnv: Registering MapOutputTracker
23/11/16 03:25:10 INFO SparkEnv: Registering BlockManagerMaster
23/11/16 03:25:10 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
23/11/16 03:25:11 INFO SparkEnv: Registering OutputCommitCoordinator
/usr/lib/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
mqtt = StructType([StructField('tcp_flags', StringType(), True), 
                   StructField('tcp_time_delta', DoubleType(), True), 
                   StructField('tcp_len', IntegerType(), True), 
                   StructField('mqtt_conack_flags', StringType(), True), 
                   StructField('mqtt_conack_flags_reserved', DoubleType(), True), 
                   StructField('mqtt_conack_flags_sp', DoubleType(), True), 
                   StructField('mqtt_conack_val', DoubleType(), True), 
                   StructField('mqtt_conflag_cleansess', DoubleType(), True), 
                   StructField('mqtt_conflag_passwd', DoubleType(), True), 
                   StructField('mqtt_conflag_qos', DoubleType(), True), 
                   StructField('mqtt_conflag_reserved', DoubleType(), True), 
                   StructField('mqtt_conflag_retain', DoubleType(), True), 
                   StructField('mqtt_conflag_uname', DoubleType(), True), 
                   StructField('mqtt_conflag_willflag', DoubleType(), True), 
                   StructField('mqtt_conflags', StringType(), True), 
                   StructField('mqtt_dupflag', DoubleType(), True), 
                   StructField('mqtt_hdrflags', StringType(), True), 
                   StructField('mqtt_kalive', DoubleType(), True), 
                   StructField('mqtt_len', DoubleType(), True), 
                   StructField('mqtt_msg', StringType(), True), 
                   StructField('mqtt_msgid', DoubleType(), True), 
                   StructField('mqtt_msgtype', DoubleType(), True), 
                   StructField('mqtt_proto_len', DoubleType(), True), 
                   StructField('mqtt_protoname', StringType(), True), 
                   StructField('mqtt_qos', DoubleType(), True), 
                   StructField('mqtt_retain', DoubleType(), True), 
                   StructField('mqtt_sub_qos', DoubleType(), True), 
                   StructField('mqtt_suback_qos', DoubleType(), True), 
                   StructField('mqtt_ver', DoubleType(), True), 
                   StructField('mqtt_willmsg', DoubleType(), True), 
                   StructField('mqtt_willmsg_len', DoubleType(), True), 
                   StructField('mqtt_willtopic', DoubleType(), True), 
                   StructField('mqtt_willtopic_len', DoubleType(), True), 
                   StructField('target', StringType(), True), 
                   StructField('train_flag', IntegerType(), False)])

In [5]:
df_train = spark.read.csv(r"gs://mqtt-bucket-14/train70_reduced.csv", header = True, inferSchema= False, schema = mqtt)
df_test = spark.read.csv(r"gs://mqtt-bucket-14/test30_reduced.csv", header = True, inferSchema= False, schema = mqtt)
df_train = df_train.withColumn('train_flag', lit(1)) #train flag is 1 if the data is training set or 0 if test set
df_test = df_test.withColumn('train_flag', lit(0))

In [6]:
df_comb = df_train.union(df_test)
df_comb.printSchema()
df_comb.show(vertical = True)

root
 |-- tcp_flags: string (nullable = true)
 |-- tcp_time_delta: double (nullable = true)
 |-- tcp_len: integer (nullable = true)
 |-- mqtt_conack_flags: string (nullable = true)
 |-- mqtt_conack_flags_reserved: double (nullable = true)
 |-- mqtt_conack_flags_sp: double (nullable = true)
 |-- mqtt_conack_val: double (nullable = true)
 |-- mqtt_conflag_cleansess: double (nullable = true)
 |-- mqtt_conflag_passwd: double (nullable = true)
 |-- mqtt_conflag_qos: double (nullable = true)
 |-- mqtt_conflag_reserved: double (nullable = true)
 |-- mqtt_conflag_retain: double (nullable = true)
 |-- mqtt_conflag_uname: double (nullable = true)
 |-- mqtt_conflag_willflag: double (nullable = true)
 |-- mqtt_conflags: string (nullable = true)
 |-- mqtt_dupflag: double (nullable = true)
 |-- mqtt_hdrflags: string (nullable = true)
 |-- mqtt_kalive: double (nullable = true)
 |-- mqtt_len: double (nullable = true)
 |-- mqtt_msg: string (nullable = true)
 |-- mqtt_msgid: double (nullable = true)
 |-

23/11/16 03:25:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/11/16 03:25:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: tcp.flags, tcp.time_delta, tcp.len, mqtt.conack.flags, mqtt.conack.flags.reserved, mqtt.conack.flags.sp, mqtt.conack.val, mqtt.conflag.cleansess, mqtt.conflag.passwd, mqtt.conflag.qos, mqtt.conflag.reserved, mqtt.conflag.retain, mqtt.conflag.uname, mqtt.conflag.willflag, mqtt.conflags, mqtt.dupflag, mqtt.hdrflags, mqtt.kalive, mqtt.len, mqtt.msg, mqtt.msgid, mqtt.msgtype, mqtt.proto_len, mqtt.protoname, mqtt.qos, mqtt.retain, mqtt.sub.qos, mqtt.suback.qos, mqtt.ver, mqtt.willmsg, mqtt.willmsg_len, mqtt.willtopic, mqtt.willtopic_len, target
 Schema: tcp_flags, tcp_time_delta, tcp_len, mqtt_conack_flags, mqtt_conack_flags_reserved, mqtt_conack_flags_sp, mqtt_conack_val, mqtt_conflag_cleansess, mqtt_conflag_passwd, mqtt_

-RECORD 0------------------------------------------
 tcp_flags                  | 0x00000018           
 tcp_time_delta             | 0.998867             
 tcp_len                    | 10                   
 mqtt_conack_flags          | 0                    
 mqtt_conack_flags_reserved | 0.0                  
 mqtt_conack_flags_sp       | 0.0                  
 mqtt_conack_val            | 0.0                  
 mqtt_conflag_cleansess     | 0.0                  
 mqtt_conflag_passwd        | 0.0                  
 mqtt_conflag_qos           | 0.0                  
 mqtt_conflag_reserved      | 0.0                  
 mqtt_conflag_retain        | 0.0                  
 mqtt_conflag_uname         | 0.0                  
 mqtt_conflag_willflag      | 0.0                  
 mqtt_conflags              | 0                    
 mqtt_dupflag               | 0.0                  
 mqtt_hdrflags              | 0x00000030           
 mqtt_kalive                | 0.0                  
 mqtt_len   

## Bonus part - Reading data into Cloud SQL PostgreSQL Instance

In [8]:
db_properties={}
#update your db username
db_properties['username']="postgres"
#update your db password
db_properties['password']="asdfghjkl@23456"
db_properties['url']= "jdbc:postgresql://34.134.0.74/postgres"
db_properties['table']= "MQTT_Reduced" #created schema in Cloud SQL Instance using gcloud CLI
db_properties['Driver'] = 'org.postgresql.Driver'


df_comb.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option('Driver', db_properties['Driver'])\
.save()

23/11/16 03:29:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: tcp.flags, tcp.time_delta, tcp.len, mqtt.conack.flags, mqtt.conack.flags.reserved, mqtt.conack.flags.sp, mqtt.conack.val, mqtt.conflag.cleansess, mqtt.conflag.passwd, mqtt.conflag.qos, mqtt.conflag.reserved, mqtt.conflag.retain, mqtt.conflag.uname, mqtt.conflag.willflag, mqtt.conflags, mqtt.dupflag, mqtt.hdrflags, mqtt.kalive, mqtt.len, mqtt.msg, mqtt.msgid, mqtt.msgtype, mqtt.proto_len, mqtt.protoname, mqtt.qos, mqtt.retain, mqtt.sub.qos, mqtt.suback.qos, mqtt.ver, mqtt.willmsg, mqtt.willmsg_len, mqtt.willtopic, mqtt.willtopic_len, target
 Schema: tcp_flags, tcp_time_delta, tcp_len, mqtt_conack_flags, mqtt_conack_flags_reserved, mqtt_conack_flags_sp, mqtt_conack_val, mqtt_conflag_cleansess, mqtt_conflag_passwd, mqtt_conflag_qos, mqtt_conflag_reserved, mqtt_conflag_retain, mqtt_conflag_uname, mqtt_conflag_willflag, mqtt_conflags, mqtt_dupflag, mqtt_hdrflags, mqtt_kalive, mqtt_len, mqtt_msg, mqt

## Reading data out of the Database

In [9]:
df_read = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['Driver'])\
    .load()

df_read.show(1, vertical=True)

-RECORD 0--------------------------------
 tcp_flags                  | 0x00000010 
 tcp_time_delta             | 7.0E-6     
 tcp_len                    | 0          
 mqtt_conack_flags          | 0          
 mqtt_conack_flags_reserved | 0.0        
 mqtt_conack_flags_sp       | 0.0        
 mqtt_conack_val            | 0.0        
 mqtt_conflag_cleansess     | 0.0        
 mqtt_conflag_passwd        | 0.0        
 mqtt_conflag_qos           | 0.0        
 mqtt_conflag_reserved      | 0.0        
 mqtt_conflag_retain        | 0.0        
 mqtt_conflag_uname         | 0.0        
 mqtt_conflag_willflag      | 0.0        
 mqtt_conflags              | 0          
 mqtt_dupflag               | 0.0        
 mqtt_hdrflags              | 0          
 mqtt_kalive                | 0.0        
 mqtt_len                   | 0.0        
 mqtt_msg                   | 0          
 mqtt_msgid                 | 0.0        
 mqtt_msgtype               | 0.0        
 mqtt_proto_len             | 0.0 

## Task 2

Mean Length of MQTT message in Training set

In [8]:
train_lens = df_read.select('mqtt_len').where(df_read.train_flag == 1)
train_lens.summary('mean').show()

+-------+------------------+
|summary|          mqtt_len|
+-------+------------------+
|   mean|31.435725201384873|
+-------+------------------+



Average length of TCP message for each value in target

In [7]:
df_read.groupby('target').mean('tcp_len').show()

+----------+------------------+
|    target|      avg(tcp_len)|
+----------+------------------+
|   slowite|3.9993479678330797|
|bruteforce|3.9871043376318873|
|     flood|13313.415986949429|
| malformed| 20.97491761259612|
|       dos|312.65759830457716|
|legitimate| 7.776101001432345|
+----------+------------------+



An interesting insight from the above analysis, the flood type as the highest average TCP message length. Since flood attack is flooding the server with a ton of messages, this is expected. 

Return top X tcp flags

In [8]:
import pandas as pd
def freq_tcp(x):
    freq_tcps_df = df_read.groupby('tcp_flags').count().sort(col('count').desc())
    return pd.DataFrame(freq_tcps_df.head(x), columns = ['TCP FLAG', 'COUNT']) #displaying a Pandas dataframe since it looked better

In [9]:
#testing the function
freq_tcp(3)

,TCP FLAG,COUNT
0,0x00000018,183076
1,0x00000010,134547
2,0x00000011,4198


In [7]:
%pip install confluent_kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 44.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install feedparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=3b4ddf4236e8e9b0a28f7f2ce72d0ac2c795082d87880b2af7d99977f4d28101
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
Note: you may need to restart the kernel to use updated packages.


Using Google News feed to find most popular targets

In [10]:
from confluent_kafka import Producer
import socket
#Initialize Your Parameters here - Keep the variable values as is for the ones you can't find on the Confluent-Kafka connection 
KAFKA_CONFIG = {
    "bootstrap.servers":"pkc-6ojv2.us-west4.gcp.confluent.cloud:9092",
    "security.protocol":"SASL_SSL",
    "sasl.mechanisms":"PLAIN",
    "sasl.username":"7I62M2AEVU5SCUGM",
    "sasl.password":"4ogkrqAz5Tqsb8YQ/+1GOrktjwps8MYmeDdl5TpIEXik7DmjtzAHvcGNRI3aft0a",
    "session.timeout.ms":"45000",
    "group.id":"python-group-1",
    'auto.offset.reset': 'smallest',
    'client.id': socket.gethostname()
}

# Update your topic name
topic_name = "mqtt_topic"
producer = Producer(KAFKA_CONFIG)


%4|1700097280.602|CONFWARN|cluster-83ba-m#producer-1| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1700097280.602|CONFWARN|cluster-83ba-m#producer-1| [thrd:app]: Configuration property session.timeout.ms is a consumer property and will be ignored by this producer instance
%4|1700097280.602|CONFWARN|cluster-83ba-m#producer-1| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


In [13]:
import feedparser
import time
# We are searching for Popular Cyber Attacks in the news
feed_url = "https://news.google.com/rss/search?q=popular+cyber+attacks"
def extract_news_feed(feed_url):
    start_time = time.time()
    print('Feed started at ' + str(start_time))
    feed = feedparser.parse(feed_url)
    articles = []
    extracted_articles = set()
    while time.time() < 300 + start_time: #running five minutes feed
        for entry in feed.entries:
            link = entry.link
            title = entry.title.encode('ascii', 'ignore').decode()
            unique_id = f'{link}-{title}'
            if unique_id in extracted_articles:
                continue
            extracted_articles.add(unique_id)
            article_data = {"title": title, "link":link}
            if article_data is not None:
                producer.produce(topic_name, key=article_data["title"], value=article_data["link"])
    print('Feed ended at ' + str(time.time()))
    print('Total time elapsed ' + str(time.time() - start_time))
    producer.flush()

    
extract_news_feed(feed_url)

Feed started at 1700097307.1046758
Feed ended at 1700097607.104775
Total time elapsed 300.00021982192993


In [19]:
from confluent_kafka import Consumer
from pyspark.sql.types import *
import string


# Clean the punctation by making a translation table that maps punctations to empty strings
translator = str.maketrans("", "", string.punctuation)


emp_RDD = spark.sparkContext.emptyRDD()
# Defining the schema of the DataFrame
columns = StructType([StructField('key', StringType(), False),
                      StructField('value', StringType(), False)])

# Creating an empty DataFrame
df = spark.createDataFrame(data=emp_RDD,
                                   schema=columns)
 
# Printing the DataFrame with no data
df.show()

consumer = Consumer(KAFKA_CONFIG)
consumer.subscribe([topic_name])

try:
    i = 0
    while i < 5:
        msg = consumer.poll(timeout=1.0)
        if msg is None:
            i = i + 1
            print("Waiting...")
            continue
        if msg is not None:
            key = msg.key().decode('utf-8').lower().translate(translator)
            cleaned_key = " ".join(key.split())
            value = msg.value().decode('utf-8')
            added_row = [[cleaned_key,value]]
            added_df = spark.createDataFrame(added_row, columns)
            df = df.union(added_df)

except KeyboardInterrupt:
    pass
finally:
    consumer.close()
    df.show()


+---+-----+
|key|value|
+---+-----+
+---+-----+

Waiting...
Waiting...
Waiting...
Waiting...
Waiting...


+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|australia says ha...|https://news.goog...|
|uk faces enduring...|https://news.goog...|
|ag phil weiser un...|https://news.goog...|
|10 things i know ...|https://news.goog...|
|iowa offers schoo...|https://news.goog...|
|tackling alert fa...|https://news.goog...|
|how to keep finan...|https://news.goog...|
|ontario governmen...|https://news.goog...|
|139 million cyber...|https://news.goog...|
|report blames chi...|https://news.goog...|
|trustwave spiderl...|https://news.goog...|
|software supply c...|https://news.goog...|
|82 of attacks sho...|https://news.goog...|
|openai attributes...|https://news.goog...|
|upfort wins 2023 ...|https://news.goog...|
|qa more stable an...|https://news.goog...|
|after canada and ...|https://news.goog...|
|exploitation of c...|https://news.goog...|
|cybersecurity tre...|https://news.goog...|
|the top 9 cyber s...|https://ne

In [20]:
from pyspark.sql.functions import *
import nltk 
nltk.download('stopwords')

stop_words = nltk.corpus.stopwords.words('english')
streamed_data = df.withColumn('word', explode(split(col('key'), ' '))) \
                .filter(~col('word').isin(stop_words)) \
                .groupBy('word') \
                .count() \
                .sort('count', ascending=False)
    
streamed_data.show()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


+-------------+-----+
|         word|count|
+-------------+-----+
|        cyber|   59|
|     security|   31|
|      attacks|   20|
|     business|   12|
|cybersecurity|   12|
|          hub|   10|
|      threats|   10|
|         2023|   10|
| cyberattacks|    9|
|       attack|    9|
|     magazine|    8|
|   ransomware|    7|
|     national|    7|
|       forbes|    7|
|       trends|    6|
|       centre|    5|
|       global|    5|
|       report|    5|
|  cyberattack|    5|
|        could|    5|
+-------------+-----+
only showing top 20 rows



In [21]:
attack_types = ['slowite', 'dos', 'legitimate', 'malformed', 'brute', 'ddos', 'denial-of-service']

In [22]:
attack_count = streamed_data.filter(col('word').isin(attack_types))
attack_count.show()

+----+-----+
|word|count|
+----+-----+
|ddos|    1|
+----+-----+

